In [19]:
import os
import pandas as pd
import path
import subprocess
from subprocess import Popen, PIPE
import json
import glob

In [24]:
DIB_dir = "DIB/"
NCGR_dir = "NCGR/"
DIB_files = os.listdir(DIB_dir)
print(DIB_files)
NCGR_files = os.listdir(NCGR_dir)
print(NCGR_files)

['missing_busco_list_MMETSP0045.tsv', 'run_MMETSP0329', 'missing_busco_list_MMETSP0121.tsv', 'run_MMETSP0717', 'missing_busco_list_MMETSP0232.tsv', 'run_MMETSP0932', 'missing_busco_list_MMETSP0169.tsv', 'run_MMETSP0045', 'run_MMETSP0439', 'run_MMETSP0232', 'missing_busco_list_MMETSP0439.tsv', 'run_MMETSP0121', 'missing_busco_list_MMETSP0717.tsv', 'run_MMETSP0169', 'missing_busco_list_MMETSP0932.tsv', 'missing_busco_list_MMETSP0329.tsv']
['missing_busco_list_MMETSP0045.tsv', '.DS_Store', 'run_MMETSP0329', 'missing_busco_list_MMETSP0121.tsv', 'run_MMETSP0717', 'missing_busco_list_MMETSP0232.tsv', 'run_MMETSP0932', 'missing_busco_list_MMETSP0169.tsv', 'run_MMETSP0045', 'run_MMETSP0439', 'run_MMETSP0232', 'missing_busco_list_MMETSP0439.tsv', 'run_MMETSP0121', 'missing_busco_list_MMETSP0717.tsv', 'run_MMETSP0169', 'missing_busco_list_MMETSP0932.tsv', 'missing_busco_list_MMETSP0329.tsv']


In [78]:
# What is in NCGR not in DIB?
# Are these assmeblies more fragmented in DIB than NCGR?
# Why are they not located in DIB?
common_dib = []
for missing_file in DIB_files:
    if missing_file.endswith(".tsv"):
        MMETSP = missing_file.split("_")[3]
        MMETSP_ID = MMETSP[:-4]
        print('=======')
        print(MMETSP_ID)
        print('=======')
        run_dir = ''.join(["run_",MMETSP_ID,"/"])
        df_dib = pd.read_csv(DIB_dir+missing_file,skiprows=3)
        df_dib.columns = ['orthodb']
        #print("Missing in DIB:",len(df_dib))
        for missing_file2 in NCGR_files:
            if missing_file2.endswith(".tsv"):
                sample = missing_file2.split("_")[3]
                if MMETSP == sample:
                    df_ncgr = pd.read_csv(NCGR_dir+missing_file2,skiprows=3)
                    df_ncgr.columns = ['orthodb']
                    #print("Missing in NCGR:",len(df_ncgr))
                    unique_dib = df_dib[~df_dib.orthodb.isin(df_ncgr.orthodb)].dropna()
                    print("Missing only in DIB (should be present in NCGR):",len(unique_dib))
                    unique_dib_list = unique_dib['orthodb'].values.tolist()
                    #print(unique_dib_list)
                    dib_list = unique_dib['orthodb'].values.tolist()
                    for i in dib_list:
                        common_dib.append(i)
                    for j in  unique_dib_list:
                        print(j)
                        dib_full_table = DIB_dir + run_dir + "full_table_" + MMETSP_ID + ".tsv"
                        ncgr_full_table = NCGR_dir + run_dir + "full_table_" + MMETSP_ID + ".tsv"
                        dib_hmm = DIB_dir + run_dir + "hmmer_output/" + j + ".out"
                        dib_hmmer_output_files = glob.glob(dib_hmm + "*")
                        ncgr_hmm = NCGR_dir + run_dir + "hmmer_output/" + j + ".out"
                        ncgr_hmmer_output_files = glob.glob(ncgr_hmm + "*")
                        # Find contig in DIB full table
                        command_parts = ["grep '",j,"' ",dib_full_table]
                        command = ''.join(command_parts)
                        s = subprocess.Popen(command, shell=True,stdout=PIPE, stderr=PIPE)
                        s.wait()
                        for line in s.stdout:  
                            line = line.decode('utf-8').split() 
                            print("DIB:",line)
                        # Find contig in NCGR full table
                        command_parts = ["grep '",j,"' ",ncgr_full_table]
                        command = ''.join(command_parts)
                        s = subprocess.Popen(command, shell=True,stdout=PIPE, stderr=PIPE)
                        s.wait()
                        for line in s.stdout:  
                            line = line.decode('utf-8').split() 
                            print("NCGR:",line)
                        for hmm_filename in dib_hmmer_output_files:
                            with open(hmm_filename, "rU") as hmmfile:
                                next(hmmfile)
                                next(hmmfile)
                                next(hmmfile)
                                line_data = next(hmmfile).split()
                                if len(line_data) > 2:
                                    contig = line_data[0]
                                    tlen = line_data[2]
                                    print("DIB contig:",contig)
                                    print("DIB contig length:",tlen)
                                    qlen = line_data[5]
                                    #print("hmm length:",qlen)
                        for hmm_filename in ncgr_hmmer_output_files:
                            with open(hmm_filename, "rU") as hmmfile:
                                next(hmmfile)
                                next(hmmfile)
                                next(hmmfile)
                                line_data = next(hmmfile).split()
                                if len(line_data) > 2:
                                    contig = line_data[0]
                                    tlen = line_data[2]
                                    print("NCGR contig:",contig)
                                    print("NCGR contig length:",tlen)
                                    qlen = line_data[5]
                                    #print("hmm length:",qlen)


MMETSP0045
Missing only in DIB (should be present in NCGR): 91
EOG0937017X
DIB: ['EOG0937017X', 'Missing']
NCGR: ['EOG0937017X', 'Fragmented', 'CAMNT_0001278613', '326.2', '340']
DIB contig: MMETSP0045-figshare3840153v7-TRINITY_DN12213_c0_g1_i1_5
DIB contig length: 76
NCGR contig: CAMNT_0001278613_4
NCGR contig length: 453
NCGR contig: CAMNT_0001285871_4
NCGR contig length: 218
NCGR contig: CAMNT_0001286537_1
NCGR contig length: 256
EOG093701EE
DIB: ['EOG093701EE', 'Missing']
NCGR: ['EOG093701EE', 'Fragmented', 'CAMNT_0001289913', '326.7', '278']
DIB contig: MMETSP0045-figshare3840153v7-TRINITY_DN6485_c0_g1_i1_1
DIB contig length: 293
DIB contig: MMETSP0045-figshare3840153v7-TRINITY_DN9250_c2_g1_i1_5
DIB contig length: 194
DIB contig: MMETSP0045-figshare3840153v7-TRINITY_DN7605_c0_g1_i1_4
DIB contig length: 262
NCGR contig: CAMNT_0001279239_1
NCGR contig length: 855
NCGR contig: CAMNT_0001305087_5
NCGR contig length: 243
NCGR contig: CAMNT_0001289913_1
NCGR contig length: 716
EOG093701

/Users/johnsolk/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:55: DeprecationWarning: 'U' mode is deprecated
/Users/johnsolk/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:68: DeprecationWarning: 'U' mode is deprecated


 ['EOG093703WD', 'Missing']
NCGR: ['EOG093703WD', 'Complete', 'CAMNT_0001299839', '543.4', '564']
DIB contig: MMETSP0045-figshare3840153v7-TRINITY_DN7713_c0_g1_i1_6
DIB contig length: 73
DIB contig: MMETSP0045-figshare3840153v7-TRINITY_DN7713_c1_g1_i1_5
DIB contig length: 121
NCGR contig: CAMNT_0001299839_6
NCGR contig length: 846
EOG093704GD
DIB: ['EOG093704GD', 'Missing']
NCGR: ['EOG093704GD', 'Complete', 'CAMNT_0001292407', '214.2', '235']
NCGR contig: CAMNT_0001306627_3
NCGR contig length: 326
NCGR contig: CAMNT_0001292407_4
NCGR contig length: 308
EOG093704Q0
DIB: ['EOG093704Q0', 'Missing']
NCGR: ['EOG093704Q0', 'Complete', 'CAMNT_0001297909', '193.3', '283']
NCGR contig: CAMNT_0001297909_3
NCGR contig length: 1435
EOG093704UY
DIB: ['EOG093704UY', 'Missing']
NCGR: ['EOG093704UY', 'Fragmented', 'CAMNT_0001305971', '183.2', '201']
DIB contig: MMETSP0045-figshare3840153v7-TRINITY_DN7487_c1_g1_i1_1
DIB contig length: 580
DIB contig: MMETSP0045-figshare3840153v7-TRINITY_DN13360_c0_g1_i

NCGR: ['EOG09370FVX', 'Complete', 'CAMNT_0001300161', '288.1', '294']
NCGR contig: CAMNT_0001300161_4
NCGR contig length: 548
NCGR contig: CAMNT_0001325215_5
NCGR contig length: 562
EOG09370FXE
DIB: ['EOG09370FXE', 'Missing']
NCGR: ['EOG09370FXE', 'Fragmented', 'CAMNT_0001310551', '67.7', '89']
DIB contig: MMETSP0045-figshare3840153v7-TRINITY_DN13701_c0_g1_i1_1
DIB contig length: 217
NCGR contig: CAMNT_0001310551_2
NCGR contig length: 397
EOG09370H3E
DIB: ['EOG09370H3E', 'Missing']
NCGR: ['EOG09370H3E', 'Complete', 'CAMNT_0001294771', '341.4', '273']
NCGR contig: CAMNT_0001294771_4
NCGR contig length: 386
NCGR contig: CAMNT_0001310673_5
NCGR contig length: 960
NCGR contig: CAMNT_0001307753_4
NCGR contig length: 375
EOG09370HKV
DIB: ['EOG09370HKV', 'Missing']
NCGR: ['EOG09370HKV', 'Complete', 'CAMNT_0001292201', '62.0', '263']
NCGR contig: CAMNT_0001333339_6
NCGR contig length: 402
NCGR contig: CAMNT_0001292201_5
NCGR contig length: 400
EOG09370J9L
DIB: ['EOG09370J9L', 'Missing']
NCGR: 

DIB: ['EOG09370U0A', 'Missing']
NCGR: ['EOG09370U0A', 'Duplicated', 'CAMNT_0001305885', '145.6', '203']
NCGR: ['EOG09370U0A', 'Duplicated', 'CAMNT_0001313317', '162.6', '214']
NCGR: ['EOG09370U0A', 'Duplicated', 'CAMNT_0001316925', '147.4', '176']
DIB contig: MMETSP0045-figshare3840153v7-TRINITY_DN11963_c0_g1_i1_2
DIB contig length: 84
NCGR contig: CAMNT_0001313317_4
NCGR contig length: 466
NCGR contig: CAMNT_0001316925_4
NCGR contig length: 471
NCGR contig: CAMNT_0001305885_2
NCGR contig length: 441
EOG09370U71
DIB: ['EOG09370U71', 'Missing']
NCGR: ['EOG09370U71', 'Duplicated', 'CAMNT_0001300011', '131.0', '208']
NCGR: ['EOG09370U71', 'Duplicated', 'CAMNT_0001312937', '130.8', '208']
NCGR: ['EOG09370U71', 'Duplicated', 'CAMNT_0001332711', '131.6', '193']
DIB contig: MMETSP0045-figshare3840153v7-TRINITY_DN5732_c1_g1_i1_1
DIB contig length: 180
DIB contig: MMETSP0045-figshare3840153v7-TRINITY_DN5732_c0_g1_i1_3
DIB contig length: 91
NCGR contig: CAMNT_0001332711_3
NCGR contig length: 336

...

NCGR: ['EOG09370ER5', 'Duplicated', 'CAMNT_0003621801', '76.1', '253']
NCGR: ['EOG09370ER5', 'Duplicated', 'CAMNT_0003628099', '69.8', '230']
NCGR contig: CAMNT_0003628099_6
NCGR contig length: 1218
NCGR contig: CAMNT_0003639181_3
NCGR contig length: 182
NCGR contig: CAMNT_0003621801_2
NCGR contig length: 982
EOG09370F1Q
DIB: ['EOG09370F1Q', 'Missing']
NCGR: ['EOG09370F1Q', 'Fragmented', 'CAMNT_0003643883', '40.8', '44']
NCGR contig: CAMNT_0003643883_4
NCGR contig length: 68
EOG09370F47
DIB: ['EOG09370F47', 'Missing']
NCGR: ['EOG09370F47', 'Complete', 'CAMNT_0003621797', '429.7', '270']
NCGR contig: CAMNT_0003621797_4
NCGR contig length: 459
EOG09370FKI
DIB: ['EOG09370FKI', 'Missing']
NCGR: ['EOG09370FKI', 'Duplicated', 'CAMNT_0003621833', '98.5', '640']
NCGR: ['EOG09370FKI', 'Duplicated', 'CAMNT_0003633083', '97.9', '651']
NCGR: ['EOG09370FKI', 'Duplicated', 'CAMNT_0003645155', '98.5', '655']
NCGR contig: CAMNT_0003621833_1
NCGR contig length: 1321
NCGR contig: CAMNT_0003633083_1
NCGR

DIB: ['EOG09370KOV', 'Missing']
NCGR: ['EOG09370KOV', 'Fragmented', 'CAMNT_0003618481', '116.2', '129']
NCGR contig: CAMNT_0003618481_5
NCGR contig length: 702
EOG09370KYZ
DIB: ['EOG09370KYZ', 'Missing']
NCGR: ['EOG09370KYZ', 'Complete', 'CAMNT_0003628385', '264.5', '199']
NCGR contig: CAMNT_0003628385_2
NCGR contig length: 367
NCGR contig: CAMNT_0003628907_4
NCGR contig length: 1255
NCGR contig: CAMNT_0003629029_6
NCGR contig length: 410
EOG09370L23
DIB: ['EOG09370L23', 'Missing']
NCGR: ['EOG09370L23', 'Duplicated', 'CAMNT_0003612343', '278.7', '216']
NCGR: ['EOG09370L23', 'Duplicated', 'CAMNT_0003650533', '281.6', '272']
NCGR: ['EOG09370L23', 'Duplicated', 'CAMNT_0003652937', '279.6', '216']
NCGR contig: CAMNT_0003652937_6
NCGR contig length: 898
NCGR contig: CAMNT_0003650533_2
NCGR contig length: 781
NCGR contig: CAMNT_0003612343_6
NCGR contig length: 1139
EOG09370L77
DIB: ['EOG09370L77', 'Missing']
NCGR: ['EOG09370L77', 'Duplicated', 'CAMNT_0003616847', '231.4', '334']
NCGR: ['EOG0

NCGR: ['EOG09370QI4', 'Duplicated', 'CAMNT_0003616045', '144.3', '188']
NCGR: ['EOG09370QI4', 'Duplicated', 'CAMNT_0003647187', '144.4', '188']
NCGR contig: CAMNT_0003616045_6
NCGR contig length: 549
NCGR contig: CAMNT_0003647187_4
NCGR contig length: 538
EOG09370QJ1
DIB: ['EOG09370QJ1', 'Missing']
NCGR: ['EOG09370QJ1', 'Duplicated', 'CAMNT_0003616611', '216.1', '184']
NCGR: ['EOG09370QJ1', 'Duplicated', 'CAMNT_0003650515', '216.6', '184']
DIB contig: MMETSP0329-figshare3840153v7-TRINITY_DN9245_c0_g2_i1_3
DIB contig length: 138
DIB contig: MMETSP0329-figshare3840153v7-TRINITY_DN9245_c0_g1_i1_3
DIB contig length: 138
NCGR contig: CAMNT_0003650515_1
NCGR contig length: 956
NCGR contig: CAMNT_0003616611_4
NCGR contig length: 1096
EOG09370QKS
DIB: ['EOG09370QKS', 'Missing']
NCGR: ['EOG09370QKS', 'Complete', 'CAMNT_0003613857', '150.3', '139']
NCGR contig: CAMNT_0003613857_3
NCGR contig length: 283
EOG09370QMM
DIB: ['EOG09370QMM', 'Missing']
NCGR: ['EOG09370QMM', 'Complete', 'CAMNT_00036101

NCGR: ['EOG09370USH', 'Duplicated', 'CAMNT_0003621113', '135.5', '152']
NCGR: ['EOG09370USH', 'Duplicated', 'CAMNT_0003644887', '135.5', '152']
NCGR contig: CAMNT_0003644887_2
NCGR contig length: 352
NCGR contig: CAMNT_0003621113_2
NCGR contig length: 352
EOG09370USJ
DIB: ['EOG09370USJ', 'Missing']
NCGR: ['EOG09370USJ', 'Complete', 'CAMNT_0003613345', '182.6', '171']
NCGR contig: CAMNT_0003613345_4
NCGR contig length: 218
EOG09370UV4
DIB: ['EOG09370UV4', 'Missing']
NCGR: ['EOG09370UV4', 'Duplicated', 'CAMNT_0003624025', '136.3', '150']
NCGR: ['EOG09370UV4', 'Duplicated', 'CAMNT_0003645967', '135.3', '150']
DIB contig: MMETSP0329-figshare3840153v7-TRINITY_DN9827_c0_g1_i1_3
DIB contig length: 545
NCGR contig: CAMNT_0003624025_3
NCGR contig length: 647
NCGR contig: CAMNT_0003645967_5
NCGR contig length: 649
EOG09370VLM
DIB: ['EOG09370VLM', 'Missing']
NCGR: ['EOG09370VLM', 'Complete', 'CAMNT_0003626081', '84.3', '113']
NCGR contig: CAMNT_0003626081_4
NCGR contig length: 308
EOG09370VTP
DIB

NCGR contig: CAMNT_0003613791_3
NCGR contig length: 206
EOG093714Q2
DIB: ['EOG093714Q2', 'Missing']
NCGR: ['EOG093714Q2', 'Complete', 'CAMNT_0003621625', '69.9', '64']
NCGR contig: CAMNT_0003621625_4
NCGR contig length: 189
EOG093715EC
DIB: ['EOG093715EC', 'Missing']
NCGR: ['EOG093715EC', 'Duplicated', 'CAMNT_0003627417', '60.2', '64']
NCGR: ['EOG093715EC', 'Duplicated', 'CAMNT_0003652487', '60.2', '64']
NCGR contig: CAMNT_0003652487_5
NCGR contig length: 527
NCGR contig: CAMNT_0003627417_3
NCGR contig length: 527
EOG09371675
DIB: ['EOG09371675', 'Missing']
NCGR: ['EOG09371675', 'Complete', 'CAMNT_0003635359', '100.4', '107']
NCGR contig: CAMNT_0003608417_3
NCGR contig length: 848
NCGR contig: CAMNT_0003635359_4
NCGR contig length: 324
EOG093716T7
DIB: ['EOG093716T7', 'Missing']
NCGR: ['EOG093716T7', 'Complete', 'CAMNT_0003611589', '94.7', '192']
NCGR contig: CAMNT_0003647311_6
NCGR contig length: 746
NCGR contig: CAMNT_0003611589_3
NCGR contig length: 835
EOG093717LU
DIB: ['EOG093717L

In [79]:
print(len(common_dib))

1382


In [80]:
occurences = {}
for i in common_dib:
    if i in occurences:
        occurences[i]+=1
    else:
        occurences[i] = 1
print('Unique orthodb:',len(occurences))
sort = [(k, occurences[k]) for k in sorted(occurences, key=occurences.get, reverse=True)]

Unique orthodb: 302


In [81]:
more_than_one = []
for s in sort:
    if int(s[1]) >= 8:
        more_than_one.append(s[0])

In [82]:
print(len(more_than_one))

5


In [83]:
print(more_than_one)

['EOG0937029K', 'EOG09370F1Q', 'EOG09370KJ3', 'EOG09370OHP', 'EOG09370S0F']


In [84]:
# busco API
# https://www.orthodb.org/?page=api
# format: http://www.orthodb.org/CMD?ARG1="value"&ARG2="value&

In [86]:
for i in more_than_one:
    command_parts = ['wget ','http://www.orthodb.org','/group?','id=',i,' -O ',i,'.orthodb']
    command = ''.join(command_parts)
    print(command)
    #s = subprocess.Popen(command, shell=True)
    #s.wait()
    eog_filename = i+'.orthodb'
    with open(eog_filename) as r:
        lines = r.readlines()
        j = json.loads(lines[0])
        data = j["data"]
        name = data["name"]
        print(name)

wget http://www.orthodb.org/group?id=EOG0937029K -O EOG0937029K.orthodb
Vps54-like
wget http://www.orthodb.org/group?id=EOG09370F1Q -O EOG09370F1Q.orthodb
apoptosis antagonizing transcription factor 
wget http://www.orthodb.org/group?id=EOG09370KJ3 -O EOG09370KJ3.orthodb
Exosome complex RNA-binding protein 1/RRP40/RRP4
wget http://www.orthodb.org/group?id=EOG09370OHP -O EOG09370OHP.orthodb
tRNA (1-methyladenosine) methyltransferase catalytic subunit Gcd14
wget http://www.orthodb.org/group?id=EOG09370S0F -O EOG09370S0F.orthodb
Ribosome biogenesis protein


In [91]:
!wget http://www.orthodb.org/group?id=EOG0937060I -O EOG0937060I.orthodb

--2018-08-31 12:29:16--  http://www.orthodb.org/group?id=EOG0937060I
Resolving www.orthodb.org... 129.194.96.125
Connecting to www.orthodb.org|129.194.96.125|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.orthodb.org/group?id=EOG0937060I [following]
--2018-08-31 12:29:16--  https://www.orthodb.org/group?id=EOG0937060I
Connecting to www.orthodb.org|129.194.96.125|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2810 (2.7K) [application/json]
Saving to: 'EOG0937060I.orthodb'

EOG0937060I.orthodb 100%[===================>]   2.74K  --.-KB/s    in 0s      

2018-08-31 12:29:17 (134 MB/s) - 'EOG0937060I.orthodb' saved [2810/2810]



In [93]:
!cat EOG0937060I.orthodb

{"status": "ok", "message": null, "data": {"molecular_function": [{"id": "GO:0004386", "count": 144, "name": "GO:0004386", "type": "GeneOntology", "description": "helicase activity"}, {"id": "GO:0005524", "count": 136, "name": "GO:0005524", "type": "GeneOntology", "description": "ATP binding"}, {"id": "GO:0003676", "count": 132, "name": "GO:0003676", "type": "GeneOntology", "description": "nucleic acid binding"}, {"id": "GO:0008026", "count": 96, "name": "GO:0008026", "type": "GeneOntology", "description": "ATP-dependent helicase activity"}, {"id": "GO:0000166", "count": 86, "name": "GO:0000166", "type": "GeneOntology", "description": "nucleotide binding"}, {"id": "GO:0016787", "count": 85, "name": "GO:0016787", "type": "GeneOntology", "description": "hydrolase activity"}], "functional_category": [{"id": "C", "description": "C: Energy production and conversion", "type": "text"}, {"id": "J", "description": "J: Translation, ribosomal structure and biogenesis", "type": "text"}, {"id": "M"

In [94]:
!wget http://www.orthodb.org/group?id=EOG090C08EI -O EOG090C08EI.orthodb

--2018-08-31 12:29:52--  http://www.orthodb.org/group?id=EOG090C08EI
Resolving www.orthodb.org... 129.194.96.125
Connecting to www.orthodb.org|129.194.96.125|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.orthodb.org/group?id=EOG090C08EI [following]
--2018-08-31 12:29:52--  https://www.orthodb.org/group?id=EOG090C08EI
Connecting to www.orthodb.org|129.194.96.125|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2038 (2.0K) [application/json]
Saving to: 'EOG090C08EI.orthodb'

EOG090C08EI.orthodb 100%[===================>]   1.99K  --.-KB/s    in 0s      

2018-08-31 12:29:53 (88.3 MB/s) - 'EOG090C08EI.orthodb' saved [2038/2038]



In [95]:
!cat EOG090C08EI.orthodb

{"status": "ok", "message": null, "data": {"molecular_function": [{"id": "GO:0030296", "count": 7, "name": "GO:0030296", "type": "GeneOntology", "description": "protein tyrosine kinase activator activity"}, {"id": "GO:0030234", "count": 7, "name": "GO:0030234", "type": "GeneOntology", "description": "enzyme regulator activity"}], "gene_architecture": {"exon_stdev_counts": 5.34, "protein_stdev_length": 152.4, "protein_median_length": 462, "exon_median_counts": 11}, "tax_id": 7898, "phyletic_profile": {"species_count": 24, "gene_count": 20, "multi_copy": 1, "single_copy": 18, "present_in": 19}, "id": "EOG090C08EI", "level_name": "Actinopterygii", "unclassified GO terms": [{"id": "GO:0003674", "count": 9, "name": "GO:0003674", "type": "GeneOntology"}, {"id": "GO:0005515", "count": 8, "name": "GO:0005515", "type": "GeneOntology"}, {"id": "GO:0005575", "count": 7, "name": "GO:0005575", "type": "GeneOntology"}, {"id": "GO:0008150", "count": 7, "name": "GO:0008150", "type": "GeneOntology"}], 